In [146]:
# Importing file with libs and functions
from custom import *
from datetime import datetime, timezone
import locale

# Importing original data

In [175]:
folder_path = "C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/bronze"

# Read source files
dataframes = read_csv_files(folder_path)

# Access each DataFrame using its file name
for file_name, dataframe in dataframes.items():
    print(f"DataFrame '{file_name}'")

DataFrame 'agencias'
DataFrame 'clientes'
DataFrame 'colaboradores'
DataFrame 'colaborador_agencia'
DataFrame 'contas'
DataFrame 'propostas_credito'
DataFrame 'transacoes'


# Transforming original data

### Agencias Table
* Drop address column
* Create agency_age ('idade agencia') column

In [88]:
agencias = dataframes['agencias']

In [89]:

# Convert date_column to datetime format and make it timezone-aware
agencias['data_abertura'] = pd.to_datetime(agencias['data_abertura']).dt.tz_localize('UTC')

# Get the current date and time in UTC
current_date_time = datetime.now(timezone.utc)

# Make the current_date_time timezone-aware
current_date_time = current_date_time.replace(tzinfo=timezone.utc)

agencias['idade_agencia'] =  (current_date_time - agencias['data_abertura']).dt.days // 365
agencias = agencias.drop('endereco',axis=1)
agencias

,cod_agencia,nome,cidade,uf,data_abertura,tipo_agencia,idade_agencia
0,7,Agência Digital,São Paulo,SP,2015-08-01 00:00:00+00:00,Digital,8
1,1,Agência Matriz,São Paulo,SP,2010-01-01 00:00:00+00:00,Física,14
2,2,Agência Tatuapé,São Paulo,SP,2010-06-14 00:00:00+00:00,Física,13
3,3,Agência Campinas,Campinas,SP,2012-03-04 00:00:00+00:00,Física,11
4,4,Agência Osasco,Osasco,SP,2013-11-06 00:00:00+00:00,Física,10
5,5,Agência Porto Alegre,Porto Alegre,RS,2013-12-01 00:00:00+00:00,Física,10
6,6,Agência Rio de Janeiro,Rio de Janeiro,RJ,2015-04-01 00:00:00+00:00,Física,8
7,8,Agência Jardins,São Paulo,SP,2018-01-09 00:00:00+00:00,Física,6
8,9,Agência Florianópolis,Florianópolis,SC,2019-10-09 00:00:00+00:00,Física,4
9,10,Agência Recife,Recife,PE,2021-10-09 00:00:00+00:00,Física,2


In [95]:
write_csv(agencias,'C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/','agencias.csv')

agencias.csv written succesfully at C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/agencias.csv


## Clientes Table
* Drop personal information
* Add age column
* Add time_as_client column
* Split endereco column to create cidade and uf columns


In [122]:
clientes = dataframes['clientes']

# Dropping personal information columns
columns_to_drop = ['primeiro_nome','ultimo_nome','email','cpfcnpj','cep']

clientes = clientes.drop(columns=columns_to_drop)

# Creating 'idade' and time as a client columns

## Convert date_column to datetime format and make it timezone-aware
clientes['data_nascimento'] = pd.to_datetime(clientes['data_nascimento']).dt.tz_localize('UTC')
clientes['data_inclusao'] = pd.to_datetime(clientes['data_inclusao'])

## Get the current date and time in UTC
current_date_time = datetime.now(timezone.utc)

## Make the current_date_time timezone-aware
current_date_time = current_date_time.replace(tzinfo=timezone.utc)

clientes['idade'] =  (current_date_time - clientes['data_nascimento']).dt.days // 365
clientes['idade_inclusao'] = (clientes['data_inclusao'] - clientes['data_nascimento']).dt.days // 365
# Creating time as client column

clientes['cliente_x_anos'] = (current_date_time - clientes['data_inclusao']).dt.days // 365

# Creating cidade and uf column
clientes['cidade/uf'] = clientes['endereco'].str.extract(r'.*?(\D+)$')
clientes[['cidade', 'uf']] = clientes['cidade/uf'].str.extract(r'(.+?) \/ (.+)$')

clientes = clientes.drop(columns=['endereco','cidade/uf'])
# 
clientes

,cod_cliente,tipo_cliente,data_inclusao,data_nascimento,idade,idade_inclusao,cliente_x_anos,cidade,uf
0,28,PF,2017-04-03 16:11:00+00:00,2006-08-11 00:00:00+00:00,17,10,6,Fernandes das Pedras,SP
1,674,PF,2021-02-10 13:27:00+00:00,1995-10-11 00:00:00+00:00,28,25,2,Nunes,AP
2,693,PF,2020-01-21 13:12:00+00:00,1948-11-19 00:00:00+00:00,75,71,4,Duarte da Praia,RN
3,743,PF,2019-05-06 11:39:00+00:00,1978-01-27 00:00:00+00:00,46,41,4,Fernandes,SP
4,769,PF,2017-01-07 14:53:00+00:00,1990-08-25 00:00:00+00:00,33,26,7,Aragão,RS
...,...,...,...,...,...,...,...,...,...
993,730,PF,2019-05-02 11:22:00+00:00,1966-03-06 00:00:00+00:00,57,53,4,Azevedo do Norte,SC
994,772,PF,2017-06-08 11:13:00+00:00,1971-07-20 00:00:00+00:00,52,45,6,Duarte,MS
995,837,PF,2014-03-18 10:30:00+00:00,1944-06-03 00:00:00+00:00,79,69,9,Nunes de da Cruz,RR
996,845,PF,2020-12-16 15:14:00+00:00,1943-12-12 00:00:00+00:00,80,77,3,Moreira,PE


In [123]:
write_csv(clientes,'C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/','clientes.csv')

clientes.csv written succesfully at C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/clientes.csv


# Contas Table
* Why there's a entry difference in clientes vs. contas?
* Answer: cod_cliente = 528 is missing in Clientes table. After creating aggregations I'll look for its values and decide to keep or not. We can assign to employee with cod_colaborador 23 to fix this.
* Does data_abertura equals data_inclusao?
* Answer: No

In [132]:
# Identify rows in contas that do not have a matching cod_cliente in clientes
extra_rows = contas[~contas['cod_cliente'].isin(clientes['cod_cliente'])]
extra_rows


,num_conta,cod_cliente,cod_agencia,cod_colaborador,tipo_conta,data_abertura,saldo_total,saldo_disponivel,data_ultimo_lancamento,equal
24,528,528,1,23,PF,2013-08-15 14:41:00 UTC,7243.5916,7380.67,2018-09-04 03:56:17 UTC,True


In [133]:
clientes[clientes['cod_cliente'] == 528]

,cod_cliente,tipo_cliente,data_inclusao,data_nascimento,idade,idade_inclusao,cliente_x_anos,cidade,uf


In [134]:

# Convert 'data_inclusao' and 'data_abertura' columns to datetime format
clientes['data_inclusao'] = pd.to_datetime(clientes['data_inclusao'])
contas['data_abertura'] = pd.to_datetime(contas['data_abertura'])

# Check if 'data_inclusao' equals 'data_abertura'
equal_dates = clientes['data_inclusao'].equals(contas['data_abertura'])

# Display the result
print("Do 'data_inclusao' and 'data_abertura' columns have equal values?", equal_dates)

Do 'data_inclusao' and 'data_abertura' columns have equal values? False


In [135]:
write_csv(contas,'C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/','contas.csv')

contas.csv written succesfully at C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/contas.csv


# Colaboradores and Colaborares_Agencia
* Add age column
* Transform address column
* Find employee accountable for client with missing data


In [138]:
colaboradores = dataframes['colaboradores']

# Creating 'idade' column

## Convert date_column to datetime format and make it timezone-aware
colaboradores['data_nascimento'] = pd.to_datetime(colaboradores['data_nascimento']).dt.tz_localize('UTC')

## Get the current date and time in UTC
current_date_time = datetime.now(timezone.utc)

## Make the current_date_time timezone-aware
current_date_time = current_date_time.replace(tzinfo=timezone.utc)

colaboradores['idade'] =  (current_date_time - colaboradores['data_nascimento']).dt.days // 365

# Creating cidade and uf columns
colaboradores['cidade/uf'] = colaboradores['endereco'].str.extract(r'.*?(\D+)$')
colaboradores[['cidade', 'uf']] = colaboradores['cidade/uf'].str.extract(r'(.+?) \/ (.+)$')

colaboradores = colaboradores.drop(columns=['endereco','cidade/uf','cep','cpf'])

colaboradores

,cod_colaborador,primeiro_nome,ultimo_nome,email,data_nascimento,idade,cidade,uf
0,14,Paulo,Dias,melissalopes@example.net,1974-04-24 00:00:00+00:00,49,Castro Paulista,PE
1,19,Luiz Fernando,Dias,pcunha@example.net,1994-07-10 00:00:00+00:00,29,Sales Verde,PB
2,32,Vitor Hugo,Dias,aragaonicolas@example.net,1986-12-18 00:00:00+00:00,37,Souza do Campo,RJ
3,88,João Gabriel,Lima,luiz-otaviorezende@example.org,1994-10-19 00:00:00+00:00,29,Moura da Mata,RR
4,39,Maria Eduarda,Melo,joao-pedro72@example.com,1996-03-01 00:00:00+00:00,27,da Rosa de da Cunha,RR
...,...,...,...,...,...,...,...,...
95,94,Melissa,Gonçalves,caldeirasamuel@example.com,1971-01-09 00:00:00+00:00,53,da Mota do Amparo,TO
96,28,Alexandre,Nascimento,olivia97@example.org,1992-01-12 00:00:00+00:00,32,Cardoso da Praia,MA
97,80,Clarice,Nascimento,lorena74@example.org,1995-12-01 00:00:00+00:00,28,Jesus,SC
98,27,Marcela,da Conceição,mcosta@example.com,1963-05-14 00:00:00+00:00,60,Correia,AM


In [140]:
# Who is the employee accountable for the client with missing information?
colaboradores[colaboradores['cod_colaborador'] == 23]

,cod_colaborador,primeiro_nome,ultimo_nome,email,data_nascimento,idade,cidade,uf
63,23,Elisa,Fogaça,qjesus@example.net,1988-08-16 00:00:00+00:00,35,Melo de Lima,ES


In [142]:
write_csv(colaboradores,'C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/','colaboradores.csv')

colaborador_agencia = dataframes['colaborador_agencia']

write_csv(colaborador_agencia,'C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/','colaborador_agencia.csv')

colaboradores.csv written succesfully at C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/colaboradores.csv
colaborador_agencia.csv written succesfully at C:/Users/abori/OneDrive/Documentos/indicium_lighthouse/silver/colaborador_agencia.csv


# Propostas_Credito Table
* Add 'saldo_devedor' column by multiplying 'valor_prestacao' * 'quantidade_parcelas'
* Format numbers columns to switch from '.' as decimal separator to ','
* Add 'data_primeiro_pgto' for the date when the payment starts considering 'carencia'
* Add 'data_ult_pgto' for the last date of payments
* Add 'saldo_proposta' which calculates 'saldo_devedor' + 'valor_entrada' - 'valor_financiamento' to point the profit made by the bank in that operation

For the transformations mentioned above I've assumed (after manual analysis) that the interest fee is only applied to 'valor_proposta' and 'valor_entrada' is a down payment made by the client. Therefore, the amount received by the bank is 'saldo_devedor' + 'valor_entrada', and they lent 'valor_financiamento'


# Pagamentos_Credito Table
* In this table we use information from Propostas_Credito to create a flow of receivables from credit operations.
* If a credit proposal has 34 prestacoes, it will write 35 lines in this table. 1 for the 'valor_entrada' and the rest for 'prestacoes'.
* The 'valor_entrada' is received at the same date as 'data_entrada_proposta'.
* The next payment is at 'data_primeiro_pgto' and it ends at 'data_ult_pgto'

This table will look like this:
* cod_proposta
* cod_cliente
* data_pgto 
* tipo_pgto - Entrada or Prestacao
* num_pgto - 0 when referring to Entrada and 1+n when referring to Prestacao



# Transacoes Table
* Add tipo_transacao column
* Format valor_transacao column